In [1]:
from maddpg.maddpg import MADDPG
from maddpg.buffer import MultiAgentReplayBuffer
from torch.utils.tensorboard import SummaryWriter
from PIL import Image, ImageDraw
import numpy as np
from tools.scopa_env import *
from tools.scopone_scientifico_sim import *
import datetime
import os
from tqdm import tqdm

In [2]:
scenario = f'testMADDPG_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}'

In [3]:
env = ScopaEnv()
n_agents = 4
actor_dims = [120 for _ in range(n_agents)]
critic_dims = sum(actor_dims)

In [4]:
n_actions = 40
maddpg_agents = MADDPG(actor_dims, critic_dims, n_agents, n_actions, 
                           fc1=64, fc2=64,  
                           alpha=0.01, beta=0.01, scenario=scenario,
                           chkpt_dir='tmp/maddpg/')

memory = MultiAgentReplayBuffer(1000000, critic_dims, actor_dims, 
                        n_actions, n_agents, batch_size=1024)

In [5]:
writer = SummaryWriter(log_dir=f'runs/{scenario}')

In [6]:
# Preload card images for tensorboard:)
def preload_card_images(image_folder = '/workspace/ScopaAI_ToM/res/cards', scale_factor=0.1):
    """
    Preload all card images and scale them down dramatically to save resources.
    Args:
        image_folder (str): Path to folder containing card images.
        scale_factor (float): Factor by which to scale down images.
    Returns:
        dict: Dictionary mapping card indices to scaled-down images.
    """
    card_images = {}
    for filename in os.listdir(image_folder):
        if filename.endswith(".png"):
            # Extract rank and suit from filename
            card_name = filename.split(".")[0]  # Remove extension
            rank, suit = card_name.split("_of_")
            
            # Map suit to its corresponding value
            suit_values = {"diamonds": 30, "clubs": 20, "spades": 10, "hearts": 0}
            suit_value = suit_values[suit]

            if rank == "jack":
                rank = 8
            elif rank == "queen":
                rank = 9
            elif rank == "king":
                rank = 10
            elif rank == "ace":
                rank = 1
            
            # Calculate card index based on the new rule
            card_index = int(rank) + suit_value - 1
            
            # Load and scale down image
            image = Image.open(os.path.join(image_folder, filename))
            new_size = (int(image.width * scale_factor), int(image.height * scale_factor))
            scaled_image = image.resize(new_size)
            
            # Add to dictionary
            card_images[card_index] = scaled_image
    return card_images

card_images = preload_card_images()

In [7]:
def local_state_to_image(state, action, score = None, player = None):
    if state is None:
        return None

    if len(state.flatten()) != 120:
        raise ValueError('Local State must be 120 elements, received: ', len(state.flatten()))

    # Initialize image sections
    sections = {
        "Hand": state[:40],
        "Table": state[40:80],
        "Captures": state[80:120]
    }

    # Dimensions and image settings
    card_width, card_height = next(iter(card_images.values())).size
    card_width = int(card_width)
    card_height = int(card_height)
    row_width = 10  # Number of cards per row
    section_padding = 50  # Padding for the section labels
    section_spacing = 20  # Space between sections
    section_height = card_height + section_padding  # Total height of each section (cards + label)
    combined_height = (section_height * len(sections)) + (section_spacing * (len(sections) - 1))  # Total height
    combined_width = card_width * row_width  # Total width of the final image

    # Create the combined image with a black background
    combined_image = Image.new("RGBA", (combined_width, combined_height), (0, 0, 0, 255))
    draw = ImageDraw.Draw(combined_image)

    image_info_x = combined_width - 200
    image_info_y = 10

    if score is not None and player is not None:
        draw.text((image_info_x, image_info_y), f'Player {player.__hash__()}|Side {player.side}|Score{score}', fill="white")

    # Process each section (Hand, Table, Captures)
    y_offset = 0
    for section_name, section_state in sections.items():
        # Draw section label
        draw.text((10, y_offset + 10), section_name, fill="white")

        # Place cards in the current section
        active_indices = np.where(section_state != 0)[0]
        for i, index in enumerate(active_indices):
            # Calculate the correct card index for the global state
            card_index = index + {"Hand": 0, "Table": 40, "Captures": 80}[section_name]
            
            if index in card_images:
                card_image = card_images[index].resize((card_width, card_height))
                x = (i % row_width) * card_width
                y = y_offset + section_padding  # Place cards below the section label
                combined_image.paste(card_image, (x, y))

        # Update vertical offset for the next section (add spacing between sections)
        y_offset += section_height + section_spacing

    # Highlight the action card (if action is valid)
    if action in card_images:
        action_card = card_images[action].resize((card_width, card_height))
        # Find the section the action card belongs to
        if action < 40:
            action_section = "Hand"
            action_y_offset = 0
        elif action < 80:
            action_section = "Table"
            action_y_offset = section_height + section_spacing
        else:
            action_section = "Captures"
            action_y_offset = (section_height + section_spacing) * 2

        # Find the position of the action card within its section
        active_indices = np.where(sections[action_section] != 0)[0]
        action_position = active_indices.tolist().index(action % 40)  # Index of action card in active cards
        action_x = (action_position % row_width) * card_width
        action_y = action_y_offset + section_padding
        combined_image.paste(action_card, (action_x, action_y))
        # Draw a red border around the action card
        border_thickness = 2
        draw.rectangle(
            [action_x, action_y, action_x + card_width, action_y + card_height],
            outline="blue",
            width=border_thickness
        )

    return combined_image

In [8]:
s = np.ones(120)
s[:80] = 0
np.random.shuffle(s)
#local_state_to_image(s,0)

In [9]:
def log_state_to_tensorboard(state, action, player,player_index, step, score = None, extra = ''):
    image = local_state_to_image(state, action, score, player)
    image = np.array(image.convert('RGB'))
    writer.add_image(f"StateAction/P{player_index}" + extra, image.astype(np.uint8).transpose(2, 0, 1), step)

In [12]:
total_steps = 0
EPOCHS = 100
score_history = []
evaluate = False
best_score = 0

if evaluate:
    maddpg_agents.load_checkpoint()

for epoch in tqdm(range(EPOCHS)):
    obs = env.reset()
    env.playing_players = n_agents
    players = env.game.players
    score = 0
    done = [False]*n_agents

    ep_step = 0

    while not any(done):

        actions = maddpg_agents.choose_action(obs, players=players)
        #print(actions)
        new_observations = []
        new_rewards = []
        new_dones = []
        for i in range(n_agents):
            new_obs, reward, done[i], _ = env.step(actions[i], env.game.players[i], v=-6)
            if len(new_obs.flatten()) == 480:
                new_observations = new_obs
            else:
                new_observations.append(new_obs.flatten())
            new_rewards.append(reward)
            new_dones.append(done[i])
            score += reward
        i=0
        for o, a in zip(obs, actions):
            log_state_to_tensorboard(o, a, players[i], i, total_steps, score, extra='/Before')
            i+=1
        i=0

        
        for o, a in zip(new_observations, actions):
            log_state_to_tensorboard(o, -1, players[i], i, total_steps, score, extra='/After')
            i+=1

        if ep_step % 10 == 0:
            maddpg_agents.learn(memory)
        
        obs = new_observations

        score += sum([reward])
        total_steps += 1
        ep_step += 1

    score_history.append(score)
    avg_score = np.mean(score_history[-100:])
    if avg_score > best_score:
        os.makedirs('/workspace/ScopaAI_ToM/tmp/maddpg/' + scenario, exist_ok=True)
        maddpg_agents.save_checkpoint()
        best_score = avg_score
    print('Episode:', epoch, 'Score:', score, 'Average Score:', avg_score)

  0%|          | 0/100 [00:00<?, ?it/s]

... saving checkpoint ...


  1%|          | 1/100 [00:11<18:47, 11.39s/it]

Episode: 4 Score: 385.0 Average Score: 385.0
... saving checkpoint ...


  2%|▏         | 2/100 [00:19<15:33,  9.52s/it]

...

Episode: 4 Score: 413.0000000000001 Average Score: 338.3576530612246


 99%|█████████▉| 99/100 [19:19<00:13, 13.34s/it]

Episode: 4 Score: 169.2 Average Score: 336.64898989898995


100%|██████████| 100/100 [19:32<00:00, 11.72s/it]

Episode: 4 Score: 538.5 Average Score: 338.6675000000001


to implement and keep track on to see if algo is getting better
- difference in scores
- times a card that can captures is not chosen
- capture size of ace
- number of scopas overtime